### Import the CNN Util and libraries needed
We have the util to make it easy to create and try new variations of the CNN model and be consistent with how we're analyzing and evaluating it.

In [ ]:
# Import necessary libraries
import cnn_utils
import tensorflow as tf
import numpy as np
from keras import layers, models, optimizers, callbacks
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50, EfficientNetB0, VGG16
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import ResNet50V2
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import smart_resize
from keras.src.legacy.preprocessing.image import ImageDataGenerator


### Load the data

In [ ]:
data_dict = cnn_utils.load_cifar10_from_tar()

### Preporcess the data

In [ ]:
data = cnn_utils.preprocess_data(data_dict)

### Let's do a quick visualization of sample images (to also ensure we still have the correct shape)

In [ ]:
cnn_utils.visualize_data_samples(data)


### Data processing for Transfer Learning
- Converts back to [0, 255]: Your data was normalized to [0, 1], but ImageNet pre-trained models expect the original [0, 255] pixel range
- Applies ImageNet preprocessing: Uses preprocess_input() which applies model-specific normalization (e.g., ResNet uses different normalization than VGG)

In [ ]:
print("Resizing only validation and test sets (training will be handled by augmentation)...")

# Only resize val and test sets (much smaller)
data['X_val'] = tf.image.resize(data['X_val'], [224, 224]).numpy()
data['X_test'] = tf.image.resize(data['X_test'], [224, 224]).numpy()

print("Val and test resizing completed!")
print(f"Shapes - Train: {data['X_train'].shape} (32x32), Val: {data['X_val'].shape} (224x224), Test: {data['X_test'].shape} (224x224)")

### Data Augmentation

Moderate geometric augmentation that applies realistic transformations to training images:
- Rotation (±15°), shifting (10% in each direction), and zooming (±10%) simulate natural camera angle and distance variations
- Horizontal flipping doubles the dataset by creating mirror images (works well for CIFAR-10 since objects like cars/planes look realistic when flipped)


In [ ]:
def create_augmentation():
    def resize_and_augment(x):
        # Resize from 32x32 to 224x224 during training
        x_resized = tf.image.resize(x, [224, 224])
        return x_resized  # Remove .numpy() - keep as tensor

    return ImageDataGenerator(
        preprocessing_function=resize_and_augment,
        rotation_range=10,
        width_shift_range=0.05,
        height_shift_range=0.05,
        horizontal_flip=True,
        zoom_range=0.05
    )

augmentation = create_augmentation()
augmentation.fit(data['X_train'])

### Let's define our CNN model (architecture)
Deeper, more sophisticated architecture for higher accuracy
Structure:
- 3 convolutional blocks (64→128→256 filters)
- BatchNormalization after each conv layer
- Progressive dropout (0.3→0.4→0.5)
- Large dense layer (512 neurons)

### Create transfer model

In [ ]:
def create_transfer_model(base_model_name='resnet50', num_classes=10):
    """
    Create transfer learning model with frozen base and custom classifier
    """
    # Choose base model
    if base_model_name == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    elif base_model_name == 'efficientnet':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    elif base_model_name == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    elif base_model_name == 'resnet50v2':
        base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the base model
    base_model.trainable = False

    # Build the complete model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model, base_model

# Create the model
model, base_model = create_transfer_model('resnet50v2')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn_utils.print_model_summary(model)

In [ ]:
callbacks = [
    ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-7)
]

### Phase 1 Training (Frozen Base)


In [ ]:
print("=== PHASE 1: Training classifier with frozen base ===")
# Train with frozen base model
history_phase1 = cnn_utils.train_model(
    model,
    data,
    augmentation=augmentation,
    epochs=20,
    batch_size=32,
    callbacks=callbacks
)


### Phase 2 Training (Fine-tuning)

In [ ]:
print("=== PHASE 2A: Partial unfreezing (last 50 layers) ===")

# Unfreeze only the last 50 layers
for layer in base_model.layers[:-50]:
    layer.trainable = False
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Recompile with higher learning rate for partial fine-tuning
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),  # Higher than before
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train with partial unfreezing
history_phase2a = cnn_utils.train_model(
    model,
    data,
    augmentation=augmentation,
    epochs=10,
    batch_size=32,
    callbacks=callbacks
)

print("=== PHASE 2B: Full fine-tuning (all layers) ===")

# Unfreeze all layers
base_model.trainable = True

# Recompile with very low learning rate for full fine-tuning
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training with all layers unfrozen
history_phase2b = cnn_utils.train_model(
    model,
    data,
    augmentation=augmentation,
    epochs=10,
    batch_size=32,
    callbacks=callbacks
)

### Combined History

In [ ]:
# Combine all three training phases
def combine_three_histories(hist1, hist2a, hist2b):
    combined = {}
    for key in hist1.history.keys():
        combined[key] = hist1.history[key] + hist2a.history[key] + hist2b.history[key]

    class CombinedHistory:
        def __init__(self, history_dict):
            self.history = history_dict

    return CombinedHistory(combined)

# Combine all training phases
combined_history = combine_three_histories(history_phase1, history_phase2a, history_phase2b)

### Let's show the evaluation result

In [ ]:
cnn_utils.evaluate_model(model, data, combined_history)